<a href="https://colab.research.google.com/github/aryaneelshivam/ArysStockAnalysis/blob/main/ArysStockAnalysis(1.0.4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tradingview_ta import TA_Handler, Interval, Exchange

google_trends_js_code = """
<!-- Your Google Trends JavaScript code goes here -->
<!-- TradingView Widget BEGIN -->
<div class="tradingview-widget-container">
  <div class="tradingview-widget-container__widget"></div>
  <div class="tradingview-widget-copyright"><a href="https://in.tradingview.com/" rel="noopener nofollow" target="_blank"><span class="blue-text">Track all markets on TradingView</span></a></div>
  <script type="text/javascript" src="https://s3.tradingview.com/external-embedding/embed-widget-stock-heatmap.js" async>
  {
  "exchanges": [],
  "dataSource": "SENSEX",
  "grouping": "sector",
  "blockSize": "market_cap_basic",
  "blockColor": "Perf.3M",
  "locale": "in",
  "symbolUrl": "",
  "colorTheme": "light",
  "hasTopBar": false,
  "isDataSetEnabled": false,
  "isZoomEnabled": true,
  "hasSymbolTooltip": true,
  "width": "100%",
  "height": "100%"
}
  </script>
</div>
<!-- TradingView Widget END -->
"""

# Function to fetch stock data based on user input
def load_stock_data(stock_symbol):
    start_date = "2022-01-01"
    end_date = pd.to_datetime("today").strftime('%Y-%m-%d')
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    return stock_data

# Function to calculate buy/sell signals
def buy_sell(stock_data):
    signalBuy = []
    signalSell = []
    position = False

    for i in range(len(stock_data)):
        if stock_data['SMA5'][i] > stock_data['SMA15'][i]:
            if position == False:
                signalBuy.append(stock_data['Adj Close'][i])
                signalSell.append(None)
                position = True
            else:
                signalBuy.append(None)
                signalSell.append(None)
        elif stock_data['SMA5'][i] < stock_data['SMA15'][i]:
            if position == True:
                signalBuy.append(None)
                signalSell.append(stock_data['Adj Close'][i])
                position = False
            else:
                signalBuy.append(None)
                signalSell.append(None)
        else:
            signalBuy.append(None)
            signalSell.append(None)
    return pd.Series([signalBuy, signalSell])

# Function to calculate buy/sell signals using EMA
def buy_sellema(stock_data):
    signalBuyema = []
    signalSellema = []
    position = False

    for i in range(len(stock_data)):
        if stock_data['EMA5'][i] > stock_data['EMA15'][i]:
            if position == False:
                signalBuyema.append(stock_data['Adj Close'][i])
                signalSellema.append(None)
                position = True
            else:
                signalBuyema.append(None)
                signalSellema.append(None)
        elif stock_data['EMA5'][i] < stock_data['EMA15'][i]:
            if position == True:
                signalBuyema.append(None)
                signalSellema.append(stock_data['Adj Close'][i])
                position = False
            else:
                signalBuyema.append(None)
                signalSellema.append(None)
        else:
            signalBuyema.append(None)
            signalSellema.append(None)
    return pd.Series([signalBuyema, signalSellema])

# Function to get recommendations data
def get_recommendations(symbol, screener, exchange, interval):
    stock = TA_Handler(
        symbol=symbol,
        screener=screener,
        exchange=exchange,
        interval=interval,
    )
    return stock.get_analysis().summary

# Dash app
app = dash.Dash(__name__)

# Define the layout with tabs
app.layout = html.Div(children=[
    dcc.Tabs([
        dcc.Tab(label='Stock Analysis', children=[
            html.H1(id='header', children='Stock Price History with Buy/Sell Signals'),
            # Text input for stock symbol
            dcc.Input(id='stock-input', type='text', value='BEL.NS'),
            dcc.Graph(
                id='price-chart',
                style={'height': 500},
            ),
            dcc.Graph(
                id='price-chartSMA',
                style={'height': 500},
            ),
            dcc.Graph(
                id='price-chartEMA',
                style={'height': 500},
            ),
            dcc.Graph(
                id='price-chartBB',
                style={'height': 500},
            ),
            dcc.Graph(
                id='volume-chart',
                style={'height': 300},
            ),
        ]),
        dcc.Tab(label='Market Analysis', children=[
            html.H1(children='Recommendations Pie Chart'),
            # Text input for stock symbol in the second tab
            dcc.Input(id='recommendations-input', type='text', value='ZOMATO'),
            dcc.Graph(
                id='recommendations-pie-chart',
                style={'height': 500},
            ),
            dcc.Markdown(children=google_trends_js_code, dangerously_allow_html=True)
        ]),
    ]),
])

# Callback to update graphs based on user input
@app.callback(
    [Output('header', 'children'),
     Output('price-chart', 'figure'),
     Output('price-chartSMA', 'figure'),
     Output('price-chartEMA', 'figure'),
     Output('price-chartBB', 'figure'),
     Output('volume-chart', 'figure')],
    [Input('stock-input', 'value')]
)
def update_graphs(stock_symbol):
    stock_data = load_stock_data(stock_symbol)

    # Calculate buy/sell signals
    stock_data['EMA5'] = stock_data['Close'].ewm(span=5).mean()
    stock_data['EMA15'] = stock_data['Close'].ewm(span=15).mean()
    stock_data['SMA5'] = stock_data['Close'].rolling(window=5).mean()
    stock_data['SMA15'] = stock_data['Close'].rolling(window=15).mean()
    rstd = stock_data['Close'].rolling(window=15).std()
    upper_band = stock_data['SMA15'] + 2 * rstd
    lower_band = stock_data['SMA15'] - 2 * rstd

    stock_data[['Buy_Signal_price', 'Sell_Signal_price']] = buy_sell(stock_data)
    stock_data[['Buy_Signal_priceEMA', 'Sell_Signal_priceEMA']] = buy_sellema(stock_data)

    # Create figures for each graph
    price_chart = go.Figure(data=[
        go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
        go.Scatter(x=stock_data.index, y=stock_data['SMA5'], mode='lines', name='SMA5'),
        go.Scatter(x=stock_data.index, y=stock_data['SMA15'], mode='lines', name='SMA15'),
        go.Scatter(x=stock_data.index, y=stock_data['EMA5'], mode='lines', name='EMA5'),
        go.Scatter(x=stock_data.index, y=stock_data['EMA15'], mode='lines', name='EMA15'),
        go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_price'], mode='markers', name='Buy SMA',
                   marker=dict(color='green', size=8)),
        go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_price'], mode='markers', name='Sell SMA',
                   marker=dict(color='red', size=8)),
        go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_priceEMA'], mode='markers', name='Buy EMA',
                   marker=dict(color='black', size=8)),
        go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_priceEMA'], mode='markers', name='Sell EMA',
                   marker=dict(color='purple', size=8)),
        go.Scatter(x=stock_data.index, y=upper_band, mode='lines', name='Upper Bollinger Band'),
        go.Scatter(x=stock_data.index, y=lower_band, mode='lines', name='Lower Bollinger Band'),
    ])

    price_chart_sma = go.Figure(data=[
        go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
        go.Scatter(x=stock_data.index, y=stock_data['SMA5'], mode='lines', name='SMA5'),
        go.Scatter(x=stock_data.index, y=stock_data['SMA15'], mode='lines', name='SMA15'),
        go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_price'], mode='markers', name='Buy SMA',
                   marker=dict(color='green', size=8)),
        go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_price'], mode='markers', name='Sell SMA',
                   marker=dict(color='red', size=8)),
    ])

    price_chart_ema = go.Figure(data=[
        go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
        go.Scatter(x=stock_data.index, y=stock_data['EMA5'], mode='lines', name='EMA5'),
        go.Scatter(x=stock_data.index, y=stock_data['EMA15'], mode='lines', name='EMA15'),
        go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_priceEMA'], mode='markers', name='Buy EMA',
                   marker=dict(color='black', size=8)),
        go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_priceEMA'], mode='markers', name='Sell EMA',
                   marker=dict(color='purple', size=8)),
    ])

    price_chart_bb = go.Figure(data=[
        go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
        go.Scatter(x=stock_data.index, y=stock_data['EMA5'], mode='lines', name='EMA5'),
        go.Scatter(x=stock_data.index, y=stock_data['EMA15'], mode='lines', name='EMA15'),
        go.Scatter(x=stock_data.index, y=upper_band, mode='lines', name='Upper Bollinger Band'),
        go.Scatter(x=stock_data.index, y=lower_band, mode='lines', name='Lower Bollinger Band'),
    ])

    volume_chart = go.Figure(data=[
        go.Scatter(x=stock_data.index, y=stock_data['Volume'], mode='lines', name='Volume', line=dict(color='purple'))
    ])

    return f'{stock_symbol} Stock Price History with Buy/Sell Signals', price_chart, price_chart_sma, price_chart_ema, price_chart_bb, volume_chart

# Callback to update recommendations pie chart
@app.callback(
    Output('recommendations-pie-chart', 'figure'),
    [Input('recommendations-input', 'value')]
)
def update_recommendations_pie_chart(stock_symbol):
    symbol = stock_symbol
    screener = "india"
    exchange = "NSE"
    interval = Interval.INTERVAL_1_MONTH
    recommendations = get_recommendations(symbol, screener, exchange, interval)

    # Convert recommendations to a Pandas DataFrame
    df = pd.DataFrame(recommendations, index=[0])

    # Extract the relevant columns for the pie chart, handling missing columns
    cols_to_plot = ['BUY', 'SELL', 'NEUTRAL', 'STRONG_BUY', 'STRONG_SELL']
    existing_cols = [col for col in cols_to_plot if col in df.columns]
    pie_data = df[existing_cols]

    # Plot the pie chart
    fig = go.Figure(data=[go.Pie(labels=pie_data.columns, values=pie_data.iloc[0])])
    fig.update_layout(title=f'Recommendations for {symbol} on {exchange} - {interval}', height=500)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>